In [1]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.data import InMemoryDataset
import pickle

In [2]:
 
with open("graphs_data", "rb") as fp:   # Unpickling
    read_graph_data = pickle.load(fp)

In [3]:
data_list = []
count = 0
for data in read_graph_data:
    if data.x.shape[0] == 0:
        continue
        
    if count < 180 :
        count +=1
        data_list.append(data)
    
    elif data.y == 1:
        data_list.append(data)
        
    

In [4]:
train_loader = DataLoader(data_list, batch_size=16,shuffle=True)
test_loader = DataLoader(data_list, batch_size=16,shuffle=False)

In [5]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
#         torch.manual_seed(12345)
#         self.conv1 = CGConv((mydataset.num_node_features, hidden_channels),mydataset.num_edge_features,batch_norm=True)
        self.conv1 = GCNConv(1, hidden_channels)
        
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.conv4 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.conv4(x, edge_index)
 

        # 2. Readout layer
#         print(x.shape)
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
#         print(x.shape)
        # 3. Apply a final classifier
#         x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(1, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (conv4): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [6]:
model = GCN(hidden_channels=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# weights = torch.tensor([.05,.95]).to(device)
criterion = torch.nn.CrossEntropyLoss()

In [7]:
def train():
    model.train()
    for batch_idx,data in enumerate(train_loader):



        data = data.to(device)
        out = model(data.x, data.edge_index,data.batch)

        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        
        optimizer.step()
        optimizer.zero_grad()
#             print(batch_idx)
        

    return loss.cpu().item()
    
def test(loader):
    model.eval()

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        


        data.edge_attr = data.edge_attr.float() 
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        

        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.

test(test_loader)

0.4915254237288136

In [8]:

for epoch in range(1, 400):
    loss = train()
#     break
    train_acc = test(test_loader)
#     test_acc = test(test_loader)
    if epoch % 5 == 0: 
        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, loss : {loss:.4f}')

Epoch: 005, Train Acc: 0.4915, loss : 0.6941
Epoch: 010, Train Acc: 0.5989, loss : 0.7033
Epoch: 015, Train Acc: 0.4915, loss : 0.6918
Epoch: 020, Train Acc: 0.6469, loss : 0.6590
Epoch: 025, Train Acc: 0.7090, loss : 0.2861
Epoch: 030, Train Acc: 0.7119, loss : 0.2157
Epoch: 035, Train Acc: 0.7938, loss : 0.4588
Epoch: 040, Train Acc: 0.7486, loss : 0.6112
Epoch: 045, Train Acc: 0.8362, loss : 0.9733
Epoch: 050, Train Acc: 0.5989, loss : 0.5685
Epoch: 055, Train Acc: 0.6271, loss : 0.3092
Epoch: 060, Train Acc: 0.6949, loss : 0.4924
Epoch: 065, Train Acc: 0.6921, loss : 0.5132
Epoch: 070, Train Acc: 0.7401, loss : 1.4025
Epoch: 075, Train Acc: 0.7486, loss : 0.7167
Epoch: 080, Train Acc: 0.7373, loss : 0.3789
Epoch: 085, Train Acc: 0.7260, loss : 0.9160
Epoch: 090, Train Acc: 0.7373, loss : 0.2722
Epoch: 095, Train Acc: 0.7627, loss : 0.0766
Epoch: 100, Train Acc: 0.7373, loss : 0.7207
Epoch: 105, Train Acc: 0.7429, loss : 0.6369
Epoch: 110, Train Acc: 0.7401, loss : 0.3092
Epoch: 115

In [9]:
import numpy as np
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([p.numel() for p in model_parameters])
params

12738